## [강의] One-hot encoding

#### 문장에서 추출한 단어들에 인덱스 붙이기

In [ ]:
from konlpy.tag import Okt

phrase = "나는 자연어 처리를 배운다"
okt = Okt()
token = okt.morphs(phrase)
print(token)

word_set = {}
index = 0
for t in token:
    if t not in word_set:
        word_set[t] = index
        index += 1

print(word_set)

#### 특정 단어를 One-hot encoding 처리하기

In [ ]:
def one_hot_encoding(word, word_set):
    vector = [0] * len(word_set)
    index = word_set[word]
    vector[index] = 1
    return vector

print(word_set)
print(one_hot_encoding("자연어", word_set))

## [강의] Bag of Words (BoW)
#### BoW는 단어들의 순서는 전혀 고려하지 않고 출현 빈도에 집중한 텍스트 데이터의 수치 표현 방법
#### BoW를 만드는 과정
- 각 단어에 고유한 정수 인덱스를 부여한다.  
- 각 인덱스의 위치에 단어 토근의 등장 횟수를 기록한 벡터를 생성한다.  
- Ont-hot은 있냐 없냐 만 나타내는 방식이었음

#### 1. 단어별 인덱싱

In [ ]:
from konlpy.tag import Okt
import re

# 전처리
text1 = "정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다."
text1 = re.sub(r"\.", "", text1)

# 형태소 분리
okt = Okt()
token = okt.morphs(text1)

# 단어별 인덱싱
bow = {}
index = 0
for t in token:
    if t not in bow:
        bow[t] = index
        index += 1
print(bow)

#### 2. 단어별 빈도를 구한 후 벡터화

In [ ]:
# 벡터화
vec = []
for w, i in bow.items():
    count = token.count(w)
    vec.append(count)

print(vec)

## [강의] 사이킷런을 이용한 특징 추출(벡터화)
#### Scikit-learn은 분류, 회귀, 군집화, 의사결정 트리 등 머신러닝 알고리즘 함수를 제공하는 라이브러리
#### 텍스트 분석과 관련한 라이브러리도 사용가능
- CountVectorizer : 텍스트에서 횟수를 기준으로 특징을 추출하는 방법 (Bag of Words와 동일)  
- TFIDFVectorizer : TF-IDF라는 값(단어의 가중치)을 사용해서 텍스트에서 특징을 추출  
- HashingVectorizer : 해시 함수를 사용하여 적은 메모리와 빠른 속도를 통해 실행 시간이 단축 됨  

### 1. CountVectorizer : 텍스트에서 횟수를 기준으로 특징을 추출하는 방법 (Bag of Words와 동일)

#### 데이터로부터 단어 사전 만들기

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

text_data = [
    "나는 배가 고프다",
    "내일 점심 뭐먹지",
    "내일 공부 해야겠다.",
    "점심 먹고 공부해야지"
]

# 단어 사전 구성
count_vectorizer = CountVectorizer()     # 객체 생성
count_vectorizer.fit(text_data)          # 단어 목록 생성
print(count_vectorizer.vocabulary_)
print(len(count_vectorizer.vocabulary_))

#### 단어 사전을 이용해서 문장을 벡터화

In [ ]:
# 이전 코드 생략
# 문장을 벡터화
sentence = [text_data[0]]       # 첫 번째 문장
vector = count_vectorizer.transform(sentence).toarray()          # transform
print(vector)


### 2. TfidfVectorizer : TF-IDF라는 특정 값을 사용해서 텍스트 데이터의 특징을 추출하는 방법
- TF(Term Frequency): 특정 문서에서 특정 단어가 등장한 횟수
- DF(Document Frequency): 특정 단어가 등장한 문서의 수
- IDF(Inverse Document Frequency): DF의 역수
- DF-IDF는 DF * IDF를 의미함
- 어떤 단어가 해당 문서에는 자주 등장하지만 다른 문서에는 많이 없는 단어일 수록 높은 값을 가지게 됨
- 조사나 지시대명사처럼 자주 등장하는 단어는 TF 가 크지만 IDF가 작아짐

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

text_data = [
    "나는 배가 고프다",
    "내일 점심 뭐먹지",
    "내일 공부 해야겠다.",
    "점심 먹고 공부해야지"
]

tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(text_data)
print(tfidf_vectorizer.vocabulary_)

sentence = [text_data[3]]
vector = tfidf_vectorizer.transform(sentence).toarray()
print(vector)

## [강의 및 실습] 유사도를 이용한 영화 추천 시스템
### TF-IDF와 코사인 유사도만으로 영화의 줄거리에 기반해서 영화를 추천하는 시스템을 구축

#### 1) 데이터 로드

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer    # TF-IDF 벡터화
from sklearn.metrics.pairwise import linear_kernel      # 코사인 유사도 계산

# 데이터 읽은 후 20000개 샘플만 읽기
data = pd.read_csv("movies_metadata.csv", low_memory=False)
data = data.iloc[:20000]
data.head()

#### 2) overview가 없는 데이터 처리

In [ ]:
data["overview"] = data["overview"].fillna("")
print(data.shape)

#### 3) TF-IDF matrix 구하기

In [ ]:
# TF-IDF
tfidf = TfidfVectorizer(stop_words="english") 
tfidf_matrix = tfidf.fit_transform(data["overview"])
print(tfidf_matrix.shape)

#### 4) 코사인 유사도 계산하기 (similarity)

In [ ]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
print(cosine_sim)

#### 5) 특정 영화 제목이 주어졌을 때 이 영화와 유사도가 가장 높은 10개 영화 제목을 출력하기

In [ ]:
# cosine similarity
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
#print(cosine_sim[0][:10])

title = "The Dark Knight Rises"
title2index = pd.Series(data=data.index, index=data["title"]).drop_duplicates()
index = title2index[title]

# index 번째 영화에 대한 유사도 값에 대해 [(0, 유사도), (1, 유사도) ..] 형태로 저장
sim_scores = list(enumerate(cosine_sim[index]))

# 정렬하기
sim_scores.sort(key=lambda x:x[1], reverse=True)
top10 = sim_scores[1:11]
top10_index = [x[0] for x in top10]
print(top10_index)
print(data["title"].iloc[top10_index])